In [31]:
# Advent of Code 2022
# https://adventofcode.com/2022/day/7

In [32]:
with open("input.txt") as f:
    input = f.read()

    lines = input.split('\n')

lines

['$ cd /',
 '$ ls',
 'dir bzcg',
 'dir hrtvrp',
 'dir jvj',
 'dir ltrqb',
 'dir msqlnht',
 'dir mvs',
 'dir nzmddp',
 'dir zjvncc',
 '$ cd bzcg',
 '$ ls',
 'dir fwmbbvj',
 '286838 hclnfzgv.gqb',
 'dir mpsthvvc',
 '76013 qgzdlv.vdh',
 '186898 znzszz',
 'dir zwmp',
 '$ cd fwmbbvj',
 '$ ls',
 'dir dhdgrhg',
 'dir fmdbzs',
 'dir hfczrwl',
 'dir hzdt',
 'dir lchzdqv',
 'dir llfhrcjr',
 '276346 mqb',
 'dir nzmddp',
 '314933 nzmddp.hsf',
 'dir stg',
 '299329 tgtdlct.qbw',
 '$ cd dhdgrhg',
 '$ ls',
 '296118 sfpqf',
 '96182 znzszz',
 '$ cd ..',
 '$ cd fmdbzs',
 '$ ls',
 '35391 bfcsgnhd.qdd',
 'dir cgvwgw',
 'dir jrzcqs',
 '28249 lbd',
 '310945 qfzr.wmv',
 '135866 tgcsgnsd.tcm',
 '$ cd cgvwgw',
 '$ ls',
 'dir nzmddp',
 '$ cd nzmddp',
 '$ ls',
 '236634 qfzr.wmv',
 '$ cd ..',
 '$ cd ..',
 '$ cd jrzcqs',
 '$ ls',
 '167809 cwhfwngj',
 '110114 mvmbff.frl',
 '289563 qgzdlv.vdh',
 'dir vtjlsltn',
 '332076 wcztrpdr.bcz',
 'dir zwpjpt',
 '$ cd vtjlsltn',
 '$ ls',
 '306424 jwj',
 '345576 qgzdlv.vdh',
 '$ 

In [33]:
##########
# Strategy for creating a directory tree:
##########
 
# Directory tree is a dictionary of dictionaries.  
# Each item in the dictionary is either a file or a directory.
# Format for each item is as follows:
# {<id>:{
#   'item_id':<id>, 
#   'item':<'dir or 'file'>, 
#   'parent_id':<id of the parent directory>,
#   'name':<name of the file or directory>,
#   'contents':[<list of ids for the contents of this directory if the item is a directory>],
#   'size':<file size if the item is a file>,
#   'depth': <how many levels down from the root is this item?>}
# }

# This structure lets you trace any item in the tree to its parent directory and its contents.


In [34]:

# set up the tree with a root directory,

# id = 1, item = directory, parent directory id = 0 (this is an arbitrary id since the root 
# directory has no actual parent directory.), 
# name = 'root', list of items contained in the root 
# directory is empty so far, size = 0, depth = 0.

tree = {1:{
    'item_id':1, 
    'item':'dir', 
    'parent_id':0, 
    'name':'root', 
    'contents':[], 
    'size':0, 
    'depth':0}}

# set root as the default item to start at
wd = tree[1]

# set next available ID to assign
next_id = 2

def cd(l):
    ''' reads in an instruction line (l) and executes CD instructions: 
    either moves to the specified directory or creates it if it doesn't
    exist yet and then moves to it.'''

    global wd
    global next_id
    
    # cd \  -- go to root directory
    if l[5:] == '\\':
        
        # working directory = root directory
        wd = tree[1]

    # cd ..  -- move up one directory level
    elif l[5:] == '..':
        # change working directory to parent directory
        wd = tree[wd['parent_id']]
    
    # cd <directry name>  -- move down one level to specified subdirectory
    else:             
        subdir = l[5:]
        # loop through the contents of the working directory
        # to see if this subdirectory already exists.
        
        # Create a list of directory names under the wd
        subdir_list = []
        for c in wd['contents']:
            subdir_list.append(tree[c]['name'])

        for s in subdir_list:
            # if subdirectory exists, move to it.
            if s == subdir:
                wd = tree[c]
                

        # if subdirectory doesn't exist:
        if subdir not in subdir_list:
            # create subdirectory
            tree[next_id]={
                'item_id':next_id, 
                'item':'dir', 
                'parent_id':wd['item_id'], 
                'name':subdir, 
                'contents':[], 
                'size':0, 
                'depth':wd['depth']+1}
            # add subdirectory to list of items in working directory
            wd['contents'].append(next_id)
            # move to subdirectory
            wd = tree[next_id]
            # update next available id since this one is taken now
            next_id += 1
            



In [35]:

# test using family tree and print results. Makes it easier to follow.
test_lines = [
    '$ cd mom', 
    '$ cd me',
    '$ cd ..', 
    '$ cd matt', 
    '$ cd abby', 
    '$ cd ..', 
    '$ cd kermit', 
    '$ cd ..', 
    '$ cd ..', 
    '$ cd amy', 
    '$ cd stets']

for l in test_lines:
    cd(l)

for i in tree:
    if i == 1:
        pass
    else:
        print(f"person':{tree[i]['name']}")
        pid = tree[i]['parent_id']
        cid = tree[i]['contents']
        print(f"depth: {tree[i]['depth']}")
        print(f"parent: {tree[pid]['name']}")
        print(f"dependents: {[tree[c]['name'] for c in cid]}")
        print('\n')
        

person':mom
depth: 1
parent: root
dependents: ['me', 'matt', 'amy']


person':me
depth: 2
parent: mom
dependents: []


person':matt
depth: 2
parent: mom
dependents: ['abby', 'kermit']


person':abby
depth: 3
parent: matt
dependents: []


person':kermit
depth: 3
parent: matt
dependents: []


person':amy
depth: 2
parent: mom
dependents: ['stets']


person':stets
depth: 3
parent: amy
dependents: []




In [36]:
##########
# Strategy for executing LS 
##########

# no action needed. LS just indicates the lines that follow
# will require action (adding them to the tree if they don't)
# already exist.

In [37]:
##########
# Strategy for acting on listed directories
##########

# check to see whether directory already exists. If it doesn't, then create it.

In [38]:
##########
# Strategy for acting on listed files
##########